In [2]:
# Importing required libraries
import urllib.request
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop

In [8]:
# 1. Download and extract the dataset
def download_and_extract_data():
    # Download horse-or-human dataset
    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    with zipfile.ZipFile('horse-or-human.zip', 'r') as zip_ref:
        zip_ref.extractall('data/horse-or-human')

    # Download validation-horse-or-human dataset
    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    with zipfile.ZipFile('validation-horse-or-human.zip', 'r') as zip_ref:
        zip_ref.extractall('data/validation-horse-or-human')

In [9]:
# 2. Prepare Image Data Generators for training and validation
def prepare_data():
    # Define directories for training and validation
    TRAINING_DIR = 'data/horse-or-human'
    VALIDATION_DIR = 'data/validation-horse-or-human'

    # Image data generator for training with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )

    # Image data generator for validation
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Flow images from directories
    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(150, 150),  # Resize images to 150x150
        batch_size=32,
        class_mode='binary'  # Binary classification: Horse or Human
    )

    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    return train_generator, validation_generator

In [10]:
# 3. Build the CNN model
def build_model():
    model = Sequential([
        # First convolutional layer with 16 filters and 3x3 kernel size
        Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),

        # Second convolutional layer with 32 filters and 3x3 kernel size
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        # Third convolutional layer with 64 filters and 3x3 kernel size
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        # Flatten the output from convolutional layers
        Flatten(),

        # Fully connected layer with 512 neurons
        Dense(512, activation='relu'),

        # Output layer with 1 neuron and sigmoid activation for binary classification
        Dense(1, activation='sigmoid')
    ])

    # Compile the model using RMSprop optimizer and binary crossentropy loss
    model.compile(
        loss='binary_crossentropy',
        optimizer=RMSprop(learning_rate=0.001),  # Corrected here
        metrics=['accuracy']
    )

    return model

In [11]:
# 4. Train the model
def train_model(model, train_generator, validation_generator):
    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=100,  # Number of batches per epoch
        epochs=10,  # Number of epochs
        validation_data=validation_generator,
        validation_steps=50  # Number of validation steps
    )

    return history

In [12]:
# 5. Main function to run the complete workflow
def main():
    # Download and extract data
    download_and_extract_data()

    # Prepare the data generators
    train_generator, validation_generator = prepare_data()

    # Build the model
    model = build_model()

    # Train the model
    history = train_model(model, train_generator, validation_generator)

    # Save the trained model
    model.save('horse_or_human_model.h5')

    # Output training and validation accuracy
    print(f"Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

if __name__ == '__main__':
    main()

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 33/100 ━━━━━━━━━━━━━━━━━━━━ 1:13 1s/step - accuracy: 0.5634 - loss: 0.9033

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 382ms/step - accuracy: 0.6250 - loss: 0.7811 - val_accuracy: 0.7773 - val_loss: 0.4761
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 38s 373ms/step - accuracy: 0.8324 - loss: 0.4108 - val_accuracy: 0.7070 - val_loss: 1.6774
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 370ms/step - accuracy: 0.8876 - loss: 0.2708 - val_accuracy: 0.6836 - val_loss: 1.8937
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 375ms/step - accuracy: 0.9052 - loss: 0.2659 - val_accuracy: 0.6641 - val_loss: 2.2266
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 38s 365ms/step - accuracy: 0.9134 - loss: 0.2537 - val_accuracy: 0.7227 - val_loss: 1.5135
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 38s 366ms/step - accuracy: 0.9387 - loss: 0.1525 - val_accuracy: 0.7109 - val_loss: 1.9848
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 406ms/step - accuracy: 0.9244 - loss: 0.2036 - val_accuracy: 0.7305 - val_loss: 1.4489
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 78s 363ms/step - accuracy: 0.9619 - loss: 0.1099 - val

Training Accuracy: 0.9426
Validation Accuracy: 0.6211
